# Setup

### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


### Install Dependencies

In [ ]:
# Needed for converting tif to png
!sudo apt-get -qq install imagemagick

### Imports & Global Variables

In [ ]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


remote_code_path = "/content/drive/MyDrive/Vessels"
test_images = "/content/drive/MyDrive/my_test/images"
test_gts = "/content/drive/MyDrive/my_test/masks"
local_code_path = "/content/"+os.path.basename(remote_code_path)
results_dir = "/content/drive/MyDrive/colonoscopy_v"
first_image_index = 1
last_image_index = 16

### Copy code into working directory

In [ ]:
os.system(f"cp -r {remote_code_path} ./")

0

# Prepare Data

### Fetch and Parse VesselNet Data

In [ ]:
# Clear the code's data directory
#os.system(f"rm {local_code_path}/DRIVE/test/images/*")
#os.system(f"rm {local_code_path}/DRIVE/test/1st_manual/*")

imgs = glob.glob(f"{test_images}/*")
gts = glob.glob(f"{test_gts}/*")

imgs.sort()
gts.sort()

for i, image in enumerate(imgs):
  print(i)
  # Create n masks (n is the number of images to train on)
  os.system(f"cp {local_code_path}/DRIVE/training/mask/20_training_mask.png {local_code_path}/DRIVE/test/mask/{i+21:02d}_test_mask.png")

  # Copy images and groundtruth into the code's data directory
  # And convert them from TIF to PNG
  #os.system(f"convert -verbose -coalesce {remote_test_images}/dataset/validate/origin/{i+20:02d}_test.tif {local_code_path}/DRIVE/test/images/{i:02d}_test.png")
  #os.system(f"convert -verbose -coalesce {remote_test_images}/dataset/validate/groundtruth/{i+20:02d}_manual1.tif {local_code_path}/DRIVE/test/1st_manual/{i:02d}_manual1.png")

  # Resize both images and groundtruth to UNet's size (512x512)
  os.system(f"ffmpeg -y -i {imgs[i-first_image_index]} -s 512x512 {local_code_path}/DRIVE/test/images/{i+21:02d}_test.jpg")
  os.system(f"ffmpeg -y -i {gts[i-first_image_index]} -s 512x512 {local_code_path}/DRIVE/test/1st_manual/{i+21:02d}_manual1.png")
  os.system(f"convert {local_code_path}/DRIVE/test/images/{i+21:02d}_test.jpg {local_code_path}/DRIVE/test/images/{i+21:02d}_test.png")
  os.system(f"rm {local_code_path}/DRIVE/test/images/{i+21:02d}_test.jpg")
  os.system(f"convert -verbose -coalesce {local_code_path}/DRIVE/test/1st_manual/{i+21:02d}_manual1.jpg {local_code_path}/DRIVE/test/1st_manual/{i+21:02d}_manual1.png")
  os.system(f"rm {local_code_path}/DRIVE/test/1st_manual/{i+21:02d}_manual1.jpg")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


### Prepare Dataset

In [ ]:
# This extracts all testing and training patches and saves them into an h5 file to be used by the network
!cd Vessels && python3 prepare_datasets_DRIVE.py

original image: 05_training.jpg
ground truth name: 05_manual1.png
border masks name: 05_training_mask.png
original image: 17_training.jpg
ground truth name: 17_manual1.png
border masks name: 17_training_mask.png
original image: 01_training.jpg
ground truth name: 01_manual1.png
border masks name: 01_training_mask.png
original image: 09_training.jpg
ground truth name: 09_manual1.png
border masks name: 09_training_mask.png
original image: 10_training.jpg
ground truth name: 10_manual1.png
border masks name: 10_training_mask.png
original image: 14_training.jpg
ground truth name: 14_manual1.png
border masks name: 14_training_mask.png
original image: 08_training.jpg
ground truth name: 08_manual1.png
border masks name: 08_training_mask.png
original image: 02_training.jpg
ground truth name: 02_manual1.png
border masks name: 02_training_mask.png
original image: 12_training.jpg
ground truth name: 12_manual1.png
border masks name: 12_training_mask.png
original image: 15_training.jpg
ground truth n

# Run Testing

In [ ]:
!cd Vessels && python3 run_testing.py

run_testing.py:15: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  config.readfp(open(r'./configuration.txt'))

1. Create directory for the results (if not already existing)

2. Run the prediction on GPU (no nohup)

the side H is not compatible with the selected stride of 5
img_h 512, patch_h 48, stride_h 5
(img_h - patch_h) MOD stride_h: 4
So the H dim will be padded with additional 1 pixels
the side W is not compatible with the selected stride of 5
img_w 512, patch_w 48, stride_w 5
(img_w - patch_w) MOD stride_w: 4
So the W dim will be padded with additional 1 pixels
new full images shape: 
(15, 1, 513, 513)

test images shape:
(15, 1, 513, 513)

test mask shape:
(15, 1, 512, 512)
test images range (min-max): 0.0 - 1.0
test masks are within 0-1

Number of patches on h : 94
Number of patches on w : 94
number of patches per image: 8836, totally for this dataset: 132540
tcmalloc: large alloc 2442977280 bytes == 0x55fc5ecb6000 @  0

In [ ]:
!rm -r Vessels/DRIVE/test && cp -r /content/drive/MyDrive/White\ Light/DRIVE/test Vessels/DRIVE/

# Binarize

In [ ]:
#!/usr/bin/python3

import cv2
import glob
import numpy as np

threshold_value = 160
output_dir = '/content/drive/MyDrive/colonoscopy_v/'


# Enumerate images
predictions = glob.glob(f'{output_dir}/colonoscopy_v_Original_GroundTruth_Prediction*')
print(len(predictions))
for i, image in enumerate(predictions):
  # Read image
  img = cv2.imread(image)

  # _, img_b = cv2.threshold(img, 250, 255, cv2.THRESH_BINARY)
  # Binarize given a threshold value
  img_b = np.zeros(img.shape, dtype="uint8")
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      if (img[i, j] >= [threshold_value, threshold_value, threshold_value]).all():
        img_b[i, j] = [255, 255, 255]

  cv2.imwrite(f'{output_dir}/predictions/{i:02d}_pred.png', img_b)
